In [1]:
import yfinance as yf
pfe=yf.Ticker("PFE")

In [2]:
import pandas as pd
import pg8000
from dotenv import load_dotenv
import os


In [21]:
from dotenv import load_dotenv
import os

# Cargo variables del archivo .env
load_dotenv()


db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_database = os.getenv("DB_DATABASE")


In [22]:
#crear conección al redshift
connection = pg8000.connect(
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port,
    database=db_database
)



Probando conexión con tabla de prueba

In [5]:
import requests

In [6]:
#url = "https://api.teleport.org/api/urban_areas/slug:san-francisco-bay-area/scores/"
url = "https://api.teleport.org/api/urban_areas/"
# conexión a la API con get
response = requests.get(url)

# Consultar si lasolicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Data en formato JSON
    data = response.json()

    # Mostrar los datos obtenidos
    print(data)
else:
    # Si la solicitud no fue exitosa, mostrar el código de estado
    print("Error al realizar la solicitud. Código de estado:", response.status_code)

{'_links': {'curies': [{'href': 'https://developers.teleport.org/api/resources/Location/#!/relations/{rel}/', 'name': 'location', 'templated': True}, {'href': 'https://developers.teleport.org/api/resources/City/#!/relations/{rel}/', 'name': 'city', 'templated': True}, {'href': 'https://developers.teleport.org/api/resources/UrbanArea/#!/relations/{rel}/', 'name': 'ua', 'templated': True}, {'href': 'https://developers.teleport.org/api/resources/Country/#!/relations/{rel}/', 'name': 'country', 'templated': True}, {'href': 'https://developers.teleport.org/api/resources/Admin1Division/#!/relations/{rel}/', 'name': 'a1', 'templated': True}, {'href': 'https://developers.teleport.org/api/resources/Timezone/#!/relations/{rel}/', 'name': 'tz', 'templated': True}], 'self': {'href': 'https://api.teleport.org/api/urban_areas/'}, 'ua:item': [{'href': 'https://api.teleport.org/api/urban_areas/slug:aarhus/', 'name': 'Aarhus'}, {'href': 'https://api.teleport.org/api/urban_areas/slug:adelaide/', 'name':

In [25]:
#Creación de lista que incluye la dupla de nombre de ciudades y los links correspondientes que (incluyen luego los scores que nos interesan).
ciudades=data['_links']['ua:item']

In [11]:
ciudades

[{'href': 'https://api.teleport.org/api/urban_areas/slug:aarhus/',
  'name': 'Aarhus'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:adelaide/',
  'name': 'Adelaide'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:albuquerque/',
  'name': 'Albuquerque'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:almaty/',
  'name': 'Almaty'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:amsterdam/',
  'name': 'Amsterdam'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:anchorage/',
  'name': 'Anchorage'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:andorra/',
  'name': 'Andorra'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:ankara/',
  'name': 'Ankara'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:asheville/',
  'name': 'Asheville'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:asuncion/',
  'name': 'Asuncion'},
 {'href': 'https://api.teleport.org/api/urban_areas/slug:athens/',
  'name': 'Athe

In [18]:
schema = "bapintor_coderhouse"
table_name = "ciudades"

#Creación de tabla (ya la cree en mi esquema)
#city es la primary key porque en esta data no existen dos registros para la misma ciudad. 
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.{table_name} (
    city VARCHAR PRIMARY KEY,
    "Housing" DECIMAL(10, 2),
    "Cost of Living" DECIMAL(10, 2),
    "Startups" DECIMAL(10, 2),
    "Venture Capital" DECIMAL(10, 2),
    "Travel Connectivity" DECIMAL(10, 2),
    "Commute" DECIMAL(10, 2),
    "Business Freedom" DECIMAL(10, 2),
    "Safety" DECIMAL(10, 2),
    "Healthcare" DECIMAL(10, 2),
    "Education" DECIMAL(10, 2),
    "Environmental Quality" DECIMAL(10, 2),
    "Economy" DECIMAL(10, 2),
    "Taxation" DECIMAL(10, 2),
    "Internet Access" DECIMAL(10, 2),
    "Leisure & Culture" DECIMAL(10, 2),
    "Tolerance" DECIMAL(10, 2),
    "Outdoors" DECIMAL(10, 2)
)
-- Aplicar la clave de ordenación a la nueva tabla
SORTKEY (city);
"""


# Ejecuta la consulta CREATE TABLE
cursor.execute(create_table_query)

# Confirma los cambios en la base de datos
connection.commit()


# Ejecuta la consulta CREATE TABLE


In [14]:
import requests


#Acá genero una lista con diccionarios sobre las categorias para cada ciudad para luego facilitar la carga de datos a la tabla creada. 
lista = []

# Iterar en cada ciudad
for ciudad in ciudades:
    ciudad_url = ciudad['href']+"scores/"
    ciudad_nombre = ciudad["name"]
    scores = []

    # Realiza la solicitud GET a la API para cada ciudad con los links anteriores
    response = requests.get(ciudad_url)

    # Comprueba si la solicitud fue exitosa (
    if response.status_code == 200:
        # Obtener los datos de respuesta en formato JSON
        ciudad_data = response.json()
        for categoria in ciudad_data['categories']:
            nombre = categoria["name"]
            score = categoria['score_out_of_10']
            scores.append({"nombre": nombre, "score": score})

        lista.append({"ciudad": ciudad_nombre, "puntajes": scores})
    else:
        # Si la solicitud no fue exitosa, mostrar el código de estado
        print("Error al realizar la solicitud. Código de estado:", response.status_code)

# lisa para ver los resultados
print(lista) 


[{'ciudad': 'Aarhus', 'puntajes': [{'nombre': 'Housing', 'score': 6.1315}, {'nombre': 'Cost of Living', 'score': 4.015000000000001}, {'nombre': 'Startups', 'score': 2.827000000000001}, {'nombre': 'Venture Capital', 'score': 2.5119999999999996}, {'nombre': 'Travel Connectivity', 'score': 3.5360000000000005}, {'nombre': 'Commute', 'score': 6.31175}, {'nombre': 'Business Freedom', 'score': 9.94}, {'nombre': 'Safety', 'score': 9.616500000000002}, {'nombre': 'Healthcare', 'score': 9.130666666666666}, {'nombre': 'Education', 'score': 5.3665}, {'nombre': 'Environmental Quality', 'score': 7.633000000000001}, {'nombre': 'Economy', 'score': 4.886500000000001}, {'nombre': 'Taxation', 'score': 5.0680000000000005}, {'nombre': 'Internet Access', 'score': 8.373}, {'nombre': 'Leisure & Culture', 'score': 3.187}, {'nombre': 'Tolerance', 'score': 9.738500000000002}, {'nombre': 'Outdoors', 'score': 4.13}]}, {'ciudad': 'Adelaide', 'puntajes': [{'nombre': 'Housing', 'score': 6.3095}, {'nombre': 'Cost of Li

In [15]:
#Para visuailizar y confirmar como se guarda el primero de los puntajes (housing)
for i in lista:
    print( i["ciudad"],i["puntajes"][0]["score"])

Aarhus 6.1315
Adelaide 6.3095
Albuquerque 7.2620000000000005
Almaty 9.282
Amsterdam 3.0529999999999995
Anchorage 5.433500000000001
Andorra 3.9690000000000003
Ankara 9.928
Asheville 5.855999999999999
Asuncion 9.226500000000001
Athens 9.052499999999998
Atlanta 4.9755
Auckland 4.72
Austin 4.3345
Baku 9.745000000000001
Bali 5.0
Baltimore 4.829999999999999
Bangkok 7.3935
Barcelona 6.3229999999999995
Beijing 6.173000000000001
Beirut 6.655500000000001
Belfast 7.532000000000001
Belgrade 9.2575
Belize City 4.6425
Bengaluru 9.810999999999998
Bergen 5.229499999999999
Berlin 6.3950000000000005
Bern 4.922500000000001
Bilbao 6.888000000000001
Birmingham 5.9035
Birmingham, AL 6.5555
Bogota 9.0275
Boise 6.9635
Bologna 7.4719999999999995
Bordeaux 7.1915
Boston 1.0
Boulder 3.704
Bozeman 2.949
Bratislava 7.874499999999999
Brighton 2.49
Brisbane 5.212
Bristol 5.215999999999999
Brno 8.329000000000002
Brussels 6.3549999999999995
Bucharest 9.186500000000002
Budapest 8.570500000000001
Buenos Aires 8.812000000

In [16]:
# Crear la consulta INSERT INTO
insert_query = f"""
INSERT INTO {schema}.{table_name} (
    city,
    "Housing",
    "Cost of Living",
    "Startups",
    "Venture Capital",
    "Travel Connectivity",
    "Commute",
    "Business Freedom",
    "Safety",
    "Healthcare",
    "Education",
    "Environmental Quality",
    "Economy",
    "Taxation",
    "Internet Access",
    "Leisure & Culture",
    "Tolerance",
    "Outdoors"
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
"""

In [17]:
#puntajes guarda del 0 al 16 cada una de las categorias para cada ciudad en el mismo orden que se creó la tabla
cursor = connection.cursor()
for i in lista:
        nombre = i["ciudad"]
        p_0=i["puntajes"][0]["score"]
        p_1=i["puntajes"][1]["score"]
        p_2=i["puntajes"][2]["score"]
        p_3=i["puntajes"][3]["score"]
        p_4=i["puntajes"][4]["score"]
        p_5=i["puntajes"][5]["score"]
        p_6=i["puntajes"][6]["score"]
        p_7=i["puntajes"][7]["score"]
        p_8=i["puntajes"][8]["score"]
        p_9=i["puntajes"][9]["score"]
        p_10=i["puntajes"][10]["score"]
        p_11=i["puntajes"][11]["score"]
        p_12=i["puntajes"][12]["score"]
        p_13=i["puntajes"][13]["score"]
        p_14=i["puntajes"][14]["score"]
        p_15=i["puntajes"][15]["score"]
        p_16=i["puntajes"][16]["score"]


        
        cursor.execute(insert_query, (nombre, p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16))

        try:
            connection.commit()  # Confirmar los cambios en la base de datos
        except Exception as e:
            connection.rollback()  # Revertir la transacción en caso de error
            print("Error:", str(e))

In [18]:
#drop_table_query = f"DROP TABLE IF EXISTS {schema}.{table_name}"
#cursor.execute(drop_table_query)
connection.commit()


In [20]:
connection.close()#cerrar conexión


InterfaceError: connection is closed